In [ ]:
import cv2 as cv
import numpy as np


img = cv.imread('cars\\01-715.jpg', cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"


img = cv.GaussianBlur(img, (17, 17), 0)


thresh = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 15, 3)

# Поиск контуров
contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)


img_color = cv.imread('cars\\01-715.jpg')
assert img_color is not None, "file could not be read, check with os.path.exists()"

# Фильтрация контуров по геометрическим признакам
possible_plate_contours = []
for cnt in contours:
    # Прямоугольник 
    rect = cv.minAreaRect(cnt)
    (x, y), (w, h), angle = rect
    
    
     # Проверка на нулевую высоту или ширину
    if h == 0 or w == 0:
        continue  # Пропускаем контур, если ширина или высота равны нулю
        
        
 
    aspect_ratio = w / h if w > h else h / w
    area = w * h
    
    # Параметры для фильтрации 
    if 2.8 < aspect_ratio < 5.8 and 2000 < area < 12000:
        possible_plate_contours.append(rect)

# Обработка найденных контуров номерного знака
for rect in possible_plate_contours:
    # Получаем параметры
    (x, y), (w, h), angle = rect
    
    # Создаем бокс для отображения контура
    box = cv.boxPoints(rect)
    box = np.int0(box)
    cv.drawContours(img_color, [box], 0, (0, 255, 0), 2)
    
    # Вырезаем и поворачиваем ROI номерного знака
    M = cv.getRotationMatrix2D((x, y), angle, 1.0)
    rotated = cv.warpAffine(img_color, M, (img_color.shape[1], img_color.shape[0]))

    # Извлекаем ROI номерного знака
    roi = rotated[int(y - h//2):int(y + h//2), int(x - w//2):int(x + w//2)]

    
    cv.imshow('License Plate ROI', roi)
    cv.waitKey(0)

    
cv.imshow('Contours', img_color)
cv.waitKey(0)
cv.destroyAllWindows()